In [1]:
# 필요한 라이브러리 임포트 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import networkx as nx 
import seaborn as sns

# 중년대 (40~60) 집단의 우울증 효과 네트워크 시각화로 비교

In [5]:
# 주성분코드, ATC 코드 약물 매핑된 데이터프레임 mi_code, atc_code 존재 
df = pd.read_csv('prescription_with_ad_2021.csv')

중년층 연령대만 필터링 해서 가져옵니다. 

In [ ]:
# 40 - 60 대 환자만 뽑아오기
middle_age = df[(df['연령대코드(5세단위)']>=9) & (df['연령대코드(5세단위)']<13)]


14339

In [24]:
middle_age

,기준년도,가입자일련번호,처방내역일련번호,일련번호,성별코드,연령대코드(5세단위),시도코드,요양개시일자,약품일반성분명코드,1회 투약량,1일투약량,총투여일수,단가,금액,데이터 공개일자,ATC코드
741,2021,629412,1054818,1,2,9,41,2021-04-21,149204ATB,1.0,1,21,110.0,2310,2022-08-11,N06AA12
742,2021,540899,1214503,1,2,11,41,2021-10-05,160501ACH,1.0,1,15,154.0,2310,2022-08-11,N07CA03
747,2021,562372,1655047,1,2,10,41,2021-03-12,149204ATB,1.0,1,21,110.0,2310,2022-08-11,N06AA12
755,2021,562372,2393808,1,2,10,41,2021-05-21,149204ATB,1.0,1,21,110.0,2310,2022-08-11,N06AA12
757,2021,629412,2482029,1,2,9,41,2021-10-14,149204ATB,1.0,1,21,110.0,2310,2022-08-11,N06AA12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122897,2021,592921,9790827,1,2,9,47,2021-02-12,451504ATB,0.5,1,3,893.0,1340,2022-08-11,N05AX12
1122898,2021,268090,8810984,1,2,10,29,2021-11-28,107501ATB,0.5,1,1,23.0,12,2022-08-11,N06AA09
1122904,2021,274574,9349623,2,1,11,43,2021-05-05,512301ATB,2.0,1,14,2034.0,56952,2022-08-11,H05BX01
1122911,2021,750125,10934119,2,2,11,11,2021-02-11,207001ATB,0.5,2,42,188.0,7896,2022-08-11,G04BD04


자주 쓸 칼럼들의 이름을 바꿔줍니다. 

#### 컬럼명 변경용 딕셔너리

In [25]:
# 컬럼 이름들을 영문으로 바꿔줍니다
cols_rename = {
    '기준년도': 'year',
    '가입자일련번호': 'user_id',
    '처방내역일련번호': 'pres_id',
    '일련번호': 'med_id',
    '성별코드': 'gender',
    '연령대코드(5세단위)': 'age_group',
    '시도코드': 'region',
    '요양개시일자': 'start_date',
    '약품일반성분명코드': 'mi_code',
    '1회 투약량': 'dose_per_time',
    '1일투약량': 'dose_per_day',
    '총투여일수': 'total_days',
    '단가': 'unit_price',
    '금액': 'total_price',
    '데이터 공개일자': 'data_release_date',
    'ATC코드': 'ATC_code'
}

In [26]:
middle_age = middle_age.rename(columns = cols_rename)

In [16]:
middle_age

,year,user_id,pres_id,med_id,gender,age_group,region,start_date,mi_code,dose_per_time,dose_per_day,total_days,unit_price,total_price,data_release_date,ATC_code
0,2021,908807,640330,1,2,13,41,2021-06-28,621901ACR,1.0,1,3,584.0,1752,2022-08-11,A02BC06
1,2021,104703,944513,1,2,13,41,2021-02-25,111501ATB,1.0,1,60,611.0,36660,2022-08-11,C10AA05
2,2021,710952,1081625,1,2,13,41,2021-10-02,415601ATB,1.0,1,3,4140.0,12420,2022-08-11,N02CC03
3,2021,856296,1707917,1,2,13,41,2021-09-28,495501ACE,1.0,1,3,363.0,1089,2022-08-11,N06AX21
4,2021,908807,2043127,1,2,13,41,2021-06-26,111001ATE,1.0,1,3,77.0,231,2022-08-11,B01AC06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122911,2021,750125,10934119,2,2,11,11,2021-02-11,207001ATB,0.5,2,42,188.0,7896,2022-08-11,G04BD04
1122912,2021,543280,10970109,4,2,10,41,2021-02-14,495401ACH,2.0,1,28,276.0,15456,2022-08-11,C05CX
1122913,2021,321849,11006121,5,2,18,41,2021-08-01,378601ATB,1.5,1,30,265.0,11925,2022-08-11,N05AH04
1122914,2021,766500,11702317,3,1,17,44,2021-05-05,227001ATB,0.5,1,21,499.0,5240,2022-08-11,N06AB06


## 만성질환 목록 및 매핑 

In [27]:
comor_mapping = pd.read_csv('mapping/comorbidities_mapping.csv')

# ATC 코드를 약품명으로 바꿔줍니다.
atc_to_name = comor_mapping.set_index('ATC코드')['주성분명'].to_dict()

# 만성질환 질병 종류를 구분해줍니다. 
class_map = comor_mapping[['ATC코드', 'comorbidity_class']].drop_duplicates(subset=['ATC코드'], keep='first')
atc_to_class = class_map.set_index('ATC코드')['comorbidity_class'].to_dict()

# 만성질환 질병 종류를 구분해줍니다. (여러 질병이 하나의 ATC 코드에 매핑되는 경우)
comor_mapping['comorbidity_classes'] = comor_mapping.groupby('ATC코드')['comorbidity_class'].transform(lambda x: ' '.join(x.unique()))
atc_to_classes = comor_mapping.set_index('ATC코드')['comorbidity_classes'].to_dict()

In [28]:
# comorbidity_classes 가 주가된것을 확인 
comor_mapping

,주성분코드_9,ATC코드,주성분명,주성분코드_5,comorbidity_class,comorbidity_classes
0,163130ASY,R03AC13,formoterol,1631A,COPD_med,COPD_med asthma_med
1,163131ASY,R03AC13,formoterol,1631A,COPD_med,COPD_med asthma_med
2,218330CSI,R03AC16,procaterol,2183C,COPD_med,COPD_med
3,225503ACR,R03AC02,Salbutamol,2255A,COPD_med,COPD_med asthma_med
4,225530CSI,R03AC02,salbutamol,2255C,COPD_med,COPD_med asthma_med
...,...,...,...,...,...,...
1249,627401ACS,L02BB04,enzalutamide,6274A,antineoplastic,antineoplastic
1250,716001ATB,L02BB05,apalutamide,7160A,antineoplastic,antineoplastic
1251,109001ATB,L02BG03,anastrozole,1090A,antineoplastic,antineoplastic
1252,182201ATB,L02BG04,letrozole,1822A,antineoplastic,antineoplastic


comor_mapping의 ATC코드 리스트를 추출해줍니다 

In [29]:
temp = comor_mapping.groupby(['ATC코드'])['comorbidity_class'].nunique()
temp = temp[temp > 1]
temp

ATC코드
C03AA03    2
C03CA      2
C03CA01    2
C03CA04    2
C07AB02    2
          ..
R03DA04    2
R03DA05    2
R03DC01    2
R03DC03    2
R03DX05    2
Name: comorbidity_class, Length: 71, dtype: int64

In [30]:
comor_ATC_codes = comor_mapping['ATC코드'].unique()
comor_ATC_codes

array(['R03AC13', 'R03AC16', 'R03AC02', 'R03AC18', 'R03AK11', 'R03AK06',
       'R03AK07', 'R03AK08', 'R03AK10', 'R03AK12', 'R03AK14', 'R03BB01',
       'R03BB04', 'R03BB05', 'R03BB06', 'R03BB07', 'R03BA02', 'R03BA05',
       'R03BA08', 'R03BA09', 'R03AL03', 'R03AL05', 'R03AL06', 'R03AL08',
       'R03AL09', 'R03AL12', 'R03AL04', 'R03DA', 'R03DA05', 'R03DA08',
       'R03DA04', 'R03DA11', 'R03DA07', 'R03DC02', 'R03DC01', 'R03DC03',
       'R03DC', 'R03DC53', 'R03DX', 'R03DX05', 'R03DX07', 'R03DX09',
       'R03DX08', 'R03DX10', 'A10AB01', 'A10AB04', 'A10AB05', 'A10AC01',
       'A10AD01', 'A10AD04', 'A10AD05', 'A10AE04', 'A10AE05', 'A10BA02',
       'A10BB01', 'A10BB07', 'A10BB09', 'A10BB12', 'A10BD02', 'A10BD07',
       'A10BD15', 'A10BF01', 'A10BF02', 'A10BG03', 'A10BH01', 'A10BH02',
       'A10BH05', 'A10BJ01', 'A10BK01', 'A10BK03', 'A10BX02', 'A10BX03',
       'C03AA03', 'C03BA11', 'C03BA10', 'C03BA08', 'C03BA04', 'C03CA',
       'C03CA01', 'C03CA04', 'C03DA01', 'C03DA05', 'C03DB01

중년층 의 dataframe에서 만성질환 처방에 해당하는 ATC코드를 추출합니다 

In [31]:
df_middle_age_comor = middle_age[middle_age['ATC_code'].isin(comor_ATC_codes)]
# 환자수 추출. 
df_middle_age_comor['user_id'].nunique()

4562

## 그래프 만들기

In [32]:
def create_graph(data):
    G = nx.Graph()
    
    # 노드를 추가합니다.
    for atc_code in data['ATC_code'].unique():
        G.add_node(atc_code)
        
    # user_id와 pres_id로 그룹화하여 동시 발생을 찾습니다.
    user_cooccurrences = {}
    for user_id, user_group in data.groupby('user_id'):
        # 각 user_id에 대해 co-occurrence를 집합으로 저장합니다.
        cooccurrences = set()
        for pres_id, pres_group in user_group.groupby('pres_id'):
            atc_codes = pres_group['ATC_code'].unique()
            for i in range(len(atc_codes)):
                for j in range(i + 1, len(atc_codes)):
                    cooccurrence = tuple(sorted((atc_codes[i], atc_codes[j])))
                    cooccurrences.add(cooccurrence)
        user_cooccurrences[user_id] = cooccurrences

    # co-occurrence를 그래프에 추가합니다.
    for cooccurrences in user_cooccurrences.values():
        for atc_code1, atc_code2 in cooccurrences:
            if G.has_edge(atc_code1, atc_code2):
                G[atc_code1][atc_code2]['weight'] += 1
            else:
                G.add_edge(atc_code1, atc_code2, weight=1)

    return G

In [33]:
G_mid = create_graph(df_middle_age_comor)

# 그래프의 노드와 엣지 수를 출력합니다.
print(f"Number of nodes: {G_mid.number_of_nodes()}")
print(f"Number of edges: {G_mid.number_of_edges()}")

Number of nodes: 150
Number of edges: 1421


In [34]:
# 그래프에 label을 추가합니다.
for node in G_mid.nodes:
    G_mid.nodes[node]['label'] = atc_to_name[node]
    G_mid.nodes[node]['comorbidity'] = atc_to_class[node]
    G_mid.nodes[node]['comorbidities'] = atc_to_classes[node]

In [35]:
# 게피파일로 저장해줌 
nx.write_gexf(G_mid, '../gephi/depression_middle_age_prescription_network.gexf')

In [15]:
# ATC 코드를 약품명으로 바꿔줍니다.
atc_to_name = comor_mapping.set_index('ATC코드')['주성분명'].to_dict()

# 만성질환 질병 종류를 구분해줍니다.
class_map = comor_mapping[['ATC코드', 'comorbidity_class']].drop_duplicates(subset=['ATC코드'], keep='first')
atc_to_class = class_map.set_index('ATC코드')['comorbidity_class'].to_dict()

# 그래프에 label을 추가합니다.
for node in G_mid.nodes:
    G_mid.nodes[node]['label'] = atc_to_name[node]
    G_mid.nodes[node]['comorbidity'] = atc_to_class[node]

In [36]:
pwd

'c:\\Users\\ylee1\\OneDrive\\바탕 화면\\새 폴더\\2024 2학기\\소넷\\프로젝트\\Drug_Coprescription_Network_Analysis\\data'

In [16]:
nx.write_gexf(G_mid, 'depression_middle_age_prescription_network.gexf')